<a href="https://colab.research.google.com/github/ApoorvaKrisna/Cat-Breed-Image-Classification---kaggle/blob/main/Assignment_Cat_breed_Image_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
!pip install -q kaggle
uploaded = files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d ma7555/cat-breeds-dataset

100% 1.92G/1.93G [00:19<00:00, 79.8MB/s]
100% 1.93G/1.93G [00:19<00:00, 105MB/s] 


In [ ]:
!unzip '/content/cat-breeds-dataset.zip'

Streaming output truncated to the last 5000 lines.
  inflating: images/Tortoiseshell/46495717_25061.jpg  
  inflating: images/Tortoiseshell/46495784_25049.jpg  
  inflating: images/Tortoiseshell/46495794_25052.jpg  
  inflating: images/Tortoiseshell/46496797_24946.jpg  
  inflating: images/Tortoiseshell/46497000_24936.jpg  
  inflating: images/Tortoiseshell/46497664_24825.jpg  
  inflating: images/Tortoiseshell/46500078_24567.jpg  
  inflating: images/Tortoiseshell/46504471_24003.jpg  
  inflating: images/Tortoiseshell/46507777_23590.jpg  
  inflating: images/Tortoiseshell/46510043_23381.jpg  
  inflating: images/Tortoiseshell/46510643_23331.jpg  
  inflating: images/Tortoiseshell/46512081_23194.jpg  
  inflating: images/Tortoiseshell/46513326_23056.jpg  
  inflating: images/Tortoiseshell/46513392_23048.jpg  
  inflating: images/Tortoiseshell/46515915_22642.jpg  
  inflating: images/Tortoiseshell/46518048_22389.jpg  
  inflating: images/Tortoiseshell/46518676_22313.jpg  
  inflating: i

In [ ]:
%ls '/content/images'

 Abyssinian/                               LaPerm/
'American Bobtail'/                       'Maine Coon'/
'American Curl'/                           Manx/
'American Shorthair'/                      Munchkin/
'American Wirehair'/                       Nebelung/
'Applehead Siamese'/                      'Norwegian Forest Cat'/
 Balinese/                                 Ocicat/
 Bengal/                                  'Oriental Long Hair'/
 Birman/                                  'Oriental Short Hair'/
 Bombay/                                  'Oriental Tabby'/
'British Shorthair'/                       Persian/
 Burmese/                                  Pixiebob/
 Burmilla/                                 Ragamuffin/
 Calico/                                   Ragdoll/
'Canadian Hairless'/                      'Russian Blue'/
 Chartreux/                               'Scottish Fold'/
 Chausie/                                 'Selkirk Rex'/
 Chinchilla/                               Sia

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
import torch
import os
import csv
import glob
from datetime import datetime
from pathlib import Path
from torchsummary import summary
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
path  = Path('/content/images')

freq = []

for dirs in path.glob("*"):
  cnt = 0
  for imgs in dirs.glob("*.jpg"):
    cnt = cnt+1
  freq.append((cnt, dirs))

freq = np.array(freq)
print(freq)
srt = sorted(freq, key = lambda x: x[0], reverse = True)
print(srt)

[[37 PosixPath('/content/images/Oriental Long Hair')]
 [119 PosixPath('/content/images/Devon Rex')]
 [2256 PosixPath('/content/images/Tiger')]
 [2477 PosixPath('/content/images/Bengal')]
 [1418 PosixPath('/content/images/Maine Coon')]
 [580 PosixPath('/content/images/Norwegian Forest Cat')]
 [84 PosixPath('/content/images/Chartreux')]
 [2888 PosixPath('/content/images/Siamese')]
 [184 PosixPath('/content/images/Havana')]
 [1870 PosixPath('/content/images/Russian Blue')]
 [118 PosixPath('/content/images/Ocicat')]
 [567 PosixPath('/content/images/British Shorthair')]
 [22 PosixPath('/content/images/Singapura')]
 [991 PosixPath('/content/images/American Bobtail')]
 [94 PosixPath('/content/images/Silver')]
 [5295 PosixPath('/content/images/American Shorthair')]
 [2669 PosixPath('/content/images/Ragdoll')]
 [53027 PosixPath('/content/images/Domestic Short Hair')]
 [344 PosixPath('/content/images/Burmese')]
 [1 PosixPath('/content/images/York Chocolate')]
 [24 PosixPath('/content/images/Chau

In [ ]:
# 10 breeds : Bombay, Calico, Burmese, Himalayan, Munchkin, ragdoll, siberian, British shorthair, Russian Blue, Dilute Calico.
breeds = ['Bombay', 'Calico', 'Burmese', 'Himalayan', 'Munchkin', 'Ragdoll', 'Siberian', 'British Shorthair', 'Russian Blue', 'Dilute Calico']


In [ ]:
X_data = []
Y_data = []

for ix in range(len(srt)):
  label = str(srt[ix][1]).split('/')[3]
  if label in breeds:
    print(label)
    for imgs in srt[ix][1].glob("*.jpg"):
      img_d = image.load_img(imgs, target_size=(96,96))
      img_a = image.img_to_array(img_d)
      img_x = img_a.flatten()
      X_data.append(img_x)
      Y_data.append(label)

Calico
Dilute Calico
Ragdoll
Russian Blue
Bombay
Himalayan
British Shorthair
Burmese
Siberian
Munchkin


In [ ]:
X_data[0]

array([172., 182., 183., ..., 133., 114.,  72.], dtype=float32)

In [ ]:
X_data = np.array(X_data)
Y_data = np.array(Y_data)

In [ ]:
print(X_data.shape, Y_data.shape)

(15654, 27648) (15654,)


In [ ]:
X_data[0]

array([172., 182., 183., ..., 133., 114.,  72.], dtype=float32)

In [ ]:
X_data = X_data.reshape(15654, 96, 96, 3)

In [ ]:
from sklearn.utils import shuffle
Xn_data, Yn_data = shuffle(X_data, Y_data)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
Yn_data = le.fit_transform(Yn_data)

print(Yn_data)

[2 1 7 ... 7 4 6]


In [ ]:
print(Xn_data.shape, Yn_data.shape)

(15654, 96, 96, 3) (15654,)


In [ ]:
from tensorflow.keras.utils import to_categorical
Y_one = to_categorical(Yn_data)
print(Y_one.shape)

(15654, 10)


In [ ]:
Xn_data = Xn_data / 255.0

X_train = Xn_data[ : 12000]
Y_train = Y_one[ : 12000]
X_test = Xn_data[12000 : ]
Y_test = Y_one[12000 : ]
print(X_train.shape, Y_test.shape)

(12000, 96, 96, 3) (3654, 10)


In [ ]:
from tensorflow.keras.applications import InceptionV3

In [ ]:
model_up = InceptionV3(include_top=False, weights='imagenet', input_shape=(96, 96, 3))

model_up.summary()

87916544/87910968 [==============================] - 1s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 47, 47, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 47, 47, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 47, 47, 32)   0           batch_normalization[0][0]        
______________________

In [ ]:
av1 = GlobalAveragePooling2D()(model_up.output)
fc1 = Dense(256, activation='relu')(av1)
dp1 = Dropout(0.25)(fc1)
fc2 = Dense(10, activation='softmax')(dp1)

model_new = Model(inputs = model_up.input, outputs = fc2)

model_new.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 47, 47, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 47, 47, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 47, 47, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
model_new.compile(loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
for ix in range(len(model_new.layers)):
  print(ix, model_new.layers[ix])

0 <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7efba6c4ecd0>
1 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7efba6c57cd0>
2 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7efba6af23d0>
3 <tensorflow.python.keras.layers.core.Activation object at 0x7efba6c5b590>
4 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7efba73f8190>
5 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7efba6ad87d0>
6 <tensorflow.python.keras.layers.core.Activation object at 0x7efba6a86f10>
7 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7efba6a8ef10>
8 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7efba6a96b50>
9 <tensorflow.python.keras.layers.core.Activation object at 0x7efba73f8150>
10 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7efba6aa1cd0>
11 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7efba6c2

In [ ]:
for ix in range(311):
  model_new.layers[ix].trainable = False

model_new.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 47, 47, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 47, 47, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 47, 47, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
hist = model_new.fit(X_train, Y_train, epochs=10, validation_split=0.2)

Epoch 1/10
300/300 [==============================] - 68s 87ms/step - loss: 1.9313 - accuracy: 0.3134 - val_loss: 16.1469 - val_accuracy: 0.2338
Epoch 2/10
300/300 [==============================] - 25s 83ms/step - loss: 1.6249 - accuracy: 0.4127 - val_loss: 2.6070 - val_accuracy: 0.3825
Epoch 3/10
300/300 [==============================] - 25s 84ms/step - loss: 1.4784 - accuracy: 0.4895 - val_loss: 1.6239 - val_accuracy: 0.4167
Epoch 4/10
300/300 [==============================] - 24s 81ms/step - loss: 1.3751 - accuracy: 0.5157 - val_loss: 1.4746 - val_accuracy: 0.5225
Epoch 5/10
300/300 [==============================] - 26s 85ms/step - loss: 1.2350 - accuracy: 0.5740 - val_loss: 1.5519 - val_accuracy: 0.5050
Epoch 6/10
300/300 [==============================] - 26s 85ms/step - loss: 1.1674 - accuracy: 0.6002 - val_loss: 1.3499 - val_accuracy: 0.5521
Epoch 7/10
300/300 [==============================] - 24s 81ms/step - loss: 1.0798 - accuracy: 0.6335 - val_loss: 3.8746 - val_accuracy

In [ ]:
def predict_classes(self, x, batch_size=32, verbose=1):
    '''Generate class predictions for the input samples
    batch by batch.
    # Arguments
        x: input data, as a Numpy array or list of Numpy arrays
            (if the model has multiple inputs).
        batch_size: integer.
        verbose: verbosity mode, 0 or 1.
    # Returns
        A numpy array of class predictions.
    '''
    proba = self.predict(x, batch_size=batch_size, verbose=verbose)
    if proba.shape[-1] > 1:
        return proba.argmax(axis=-1)
    else:
        return (proba > 0.5).astype('int32')

In [ ]:
Y_pred = predict_classes(model_new, X_test) 

115/115 [==============================] - 2s 19ms/step


In [ ]:
Y_test[:10]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
Y_test.shape

(3654, 10)

In [ ]:
Y_pred.shape

(3654,)

In [ ]:
Y_pred[:10]

array([0, 4, 8, 3, 3, 7, 7, 3, 8, 5])

In [ ]:
y_pred = []

for ix in range(len(Y_pred)):
  temp = np.zeros(10)
  temp[Y_pred[ix]] = 1
  y_pred.append(temp)

In [ ]:
y_pred = np.array(y_pred)
y_pred[:10]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [ ]:
y_test = Y_test
y_test[:10]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
acc = accuracy_score(y_test, y_pred)
print(acc)

0.5801860974274767


In [ ]:
cf_matrix = multilabel_confusion_matrix(y_test, y_pred)
print(cf_matrix)

[[[2985  227]
  [  23  419]]

 [[3524    2]
  [ 128    0]]

 [[3579    0]
  [  75    0]]

 [[2613  220]
  [ 351  470]]

 [[2685  224]
  [ 385  360]]

 [[3268   86]
  [ 221   79]]

 [[3608    0]
  [  46    0]]

 [[2417  593]
  [ 168  476]]

 [[3056  182]
  [ 100  316]]

 [[3617    0]
  [  37    0]]]


In [ ]:
f1 = f1_score(y_test, y_pred, average='samples')
print(f1)

0.5801860974274767


In [ ]:
roc_auc = roc_auc_score(y_test, y_pred,average='samples')
print(roc_auc)

0.7667700541263761
